<a href="https://colab.research.google.com/github/mmillet88/russian_therapy/blob/main/Generating_response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading libraries

In [ ]:
!pip install transformers sentencepiece datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 64.0 MB/s 
     |████████████████████████████████| 365 kB 67.7 MB/s 
     |████████████████████████████████| 596 kB 56.9 MB/s 
     |████████████████████████████████| 101 kB 13.3 MB/s 
     |████████████████████████████████| 6.6 MB 51.0 MB/s 
     |████████████████████████████████| 1.1 MB 53.3 MB/s 
     |████████████████████████████████| 140 kB 60.7 MB/s 
     |████████████████████████████████| 212 kB 72.4 MB/s 
     |████████████████████████████████| 127 kB 57.9 MB/s 
     |████████████████████████████████| 144 kB 56.3 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
     |████████████████████████████████| 271 kB 73.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib

In [ ]:
import os
import pandas as pd 
import matplotlib.pyplot as plt

from transformers import AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
from transformers import BertForSequenceClassification, AutoTokenizer
from transformers import EarlyStoppingCallback
from transformers import (
    glue_tasks_num_labels,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    GlueDataset,
    GlueDataTrainingArguments,
    TrainingArguments,
)
import torch
import torch.nn as nn
from torch.nn.functional import cross_entropy
import gc

import datasets
from datasets import load_dataset
from datasets import DatasetDict

import numpy as np
from numpy.lib.function_base import average

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=e4f5111cc5c472b0b2ef007975a81a900ccd02a2f1217f5d2503b2bff3310f6e
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


# Downloading a dataset

In [ ]:
import os
!mkdir /content/emp_chatbot
!mkdir emp_chatbot/data

In [ ]:
!ls

drive  emp_chatbot  sample_data


In [ ]:
os.chdir('/content/emp_chatbot/data')

In [ ]:
from datasets.load import load_from_disk
ds = load_from_disk("/content/drive/MyDrive/chatbot/ResponseGenerationDataset")
# define a subset of dataset for a particular emotion

In [ ]:
# check that ppl is normalised
ds['train']['ppl_Is it possible that in previous conversations you may not have always considerdother viewpoints presented?'][0]

1.0

In [ ]:
ds['train']['ppl_Did you find protocol 6 distressing?'][0]

1.0

# Selecting n-closest utterances and defining the dataset for response generation




In [ ]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
pipe = pipeline("feature-extraction", model=model, tokenizer=tokenizer)

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/678M [00:00<?, ?B/s]

In [ ]:
# querry = "Чувствую себя очень плохо. Мне очень грустно."
querry = "я переживаю - смогу ли выполнить поставленную перед собой задачу "
querry_embedding = pipe(querry)

In [ ]:
filtered = ds.filter(lambda example: example['emotion']=="joy", load_from_cache_file=False)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
filtered

DatasetDict({
    train: Dataset({
        features: ['gender', 'age', 'emotion', 'How are you feeling?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?', 'Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Would you like to attempt protocols for joy?', 'ppl_Was this emotion triggered by a specific event?', 'ppl

In [ ]:
def calculate_similarity(row, querry_embedding):
  simil = util.cos_sim(row['embeddings'][0][0], querry_embedding)
  return {'similarity': simil}

In [ ]:
simil= filtered.map(lambda x: calculate_similarity(x, querry_embedding[0][0]), load_from_cache_file=False)


  0%|          | 0/282 [00:00<?, ?ex/s]

In [ ]:
simil

DatasetDict({
    train: Dataset({
        features: ['gender', 'age', 'emotion', 'How are you feeling?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?', 'Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Would you like to attempt protocols for joy?', 'ppl_Was this emotion triggered by a specific event?', 'ppl

In [ ]:
sorted_dataset = simil.sort('similarity', load_from_cache_file=False, reverse=True)
top_10_similar = sorted_dataset['train'].select([0,1,2,3,4])

In [ ]:
top_10_similar['How are you feeling?']

['Отлично. Я воодушевлен и открыт для новых впечатлений. Хочу отправиться в путешествие или изучить новое направление для себя.',
 'Изумительно. Выдали зарплату, наконец то смогу поесть',
 'Сегодня у меня выдался отличный день, многое успела сделать, что давно откладывалось.',
 'Чувство нужности, вдохновлена',
 'Я чувствую себя очень хорошо, наконец то я спокоен, я могу радоваться мелочам, проблемы отступили.']

# Novelty


In [ ]:
import nltk
from nltk.util import ngrams
 
# Function to generate n-grams from sentences.
def extract_ngrams(data, num):
    n_grams = ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]


In [ ]:
past_sentences = dict()

In [ ]:
def generate_N_grams(text,ngram=1):
  words=[word for word in text.split(" ")] 
  # print("Sentence after removing stopwords:",words)
  temp=zip(*[words[i:] for i in range(0,ngram)])
  ans=[' '.join(ngram) for ngram in temp]
  return ans

In [ ]:
def calculate_simil(n_split_1, n_split_2, n_gram):
  candidate_response = generate_N_grams(n_split_1, n_gram)
  past_response = generate_N_grams(n_split_2, n_gram)
  common_n_grams = set(candidate_response).intersection(set(past_response))
  score = len(common_n_grams)/len(candidate_response)
  return score

In [ ]:
def generate_n_gram(sentence, max_n_gram):
  n_gram_list = [generate_N_grams(sentence, n_gram) for n_gram in range(1, max_n_gram+1)]
  return n_gram_list

In [ ]:
def generate_weights(coefficient_increase, number_of_weights):
  coefficients = [coefficient_increase**i for i in range(number_of_weights)]
  first_weight = 1 / sum(coefficients) 
  weights = [first_weight*coefficient_increase**i for i in range(number_of_weights)]
  return weights

In [ ]:
def novelty_score(sent_1_n_grams, sent_2_n_grams):
  common_n_grams = [set(sent_1_n_grams[i]).intersection(set(sent_2_n_grams[i])) for i in range(len(sent_1_n_grams))]
  weights = generate_weights(coefficient_increase=1.1, number_of_weights=len(sent_1_n_grams))
  print(weights)
  scores = [len(common_n_grams[i])/min(len(sent_1_n_grams[i]), len(sent_2_n_grams[i])) for i in range(len(common_n_grams))]
  weighted_scores = [weight*score for weight, score in zip(weights, scores)]
  return sum(weighted_scores)

In [ ]:
def calculate_novelty(sentence):
  new_sentence_n_grams = generate_n_gram(sentence, max_n_gram=3)
  similarity = sum([novelty_score(new_sentence_n_grams, past_sentences[i]) for i in past_sentences])/len(past_sentences)
  novelty = 1 - similarity
  return novelty

In [ ]:
def score(row, prompt, novelty_weight=0.5, fluency_weight=0.25, empathy_weight=0.25):
  novelty_score = calculate_novelty(row[prompt])
  fluency_score = row["ppl_"+prompt]
  empathy_score = row["emp_"+prompt]
  weighted_score = novelty_weight*novelty_score + fluency_weight*fluency_score + empathy_weight*empathy_score
  print(f"{row[prompt]} has a novelty score {novelty_score} and fluency score {fluency_score} amd empathy_score{empathy_score} ")
  return {"score": weighted_score}

In [ ]:
top_10_similar["Would you like to attempt protocols for joy?"]


['Я очень рад, что вы чувствуете себя прекрасно. В связи с этим я бы посоветовал вам изучить программу дистанционного обучения для того направления которое вам будет интересно.',
 'Я очень рад слышать что у вас сегодня все отлично. У меня есть для вас интересное предложения, которое однозначно пойдет вам на пользу',
 'Рада, что вы на такой позитивной ноте сегодня. Хотелось бы посоветовать вам одно упражнение.',
 'Воспользуйтесь методикой из программы',
 'Я рад, что у вас все наладилось после той ситуации. Я могу вам посоветовать одну из дыхательных практик на случай тревоги.']

In [ ]:
df = top_10_similar.to_pandas()
df.head(20)

,gender,age,emotion,How are you feeling?,Was this emotion triggered by a specific event?,Was the event recent?,Did you find protocol 6 distressing?,Did you find protocol 11 distressing?,Is it ok to ask additional questions?,Have you strongly felt or expressed any of the following emotions towards someone?,...,emp_Have you strongly felt or expressed any of the following emotions towards someone?,emp_Do you believe you should be the savior of someone else?,emp_Do you feel that you are trying to control someone?,"emp_Do you see yourself as the victim, blaming someone else for how negative you feel?",emp_Are you always blaming and accusing yourself for when something goes wrong?,emp_Is it possible that in previous conversations you may not have always considerdother viewpoints presented?,emp_Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?,emp_Would you like to attempt protocols for joy?,embeddings,similarity
0,male,25-29,joy,Отлично. Я воодушевлен и открыт для новых впеч...,,,,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,"[[[0.21158216893672943, -0.3420383036136627, -...",[[0.8142665]]
1,male,40-49,joy,"Изумительно. Выдали зарплату, наконец то смогу...",,,,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,"[[[-0.4679758846759796, -0.8040412068367004, 0...",[[0.7988922]]
2,female,30-39,joy,"Сегодня у меня выдался отличный день, многое у...",,,,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,"[[[0.21976745128631592, -0.9977736473083496, 0...",[[0.7938866]]
3,female,30-39,joy,"Чувство нужности, вдохновлена",,,,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.33,"[[[-0.31636789441108704, -1.3595733642578125, ...",[[0.7910531]]
4,female,18-24,joy,"Я чувствую себя очень хорошо, наконец то я спо...",,,,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,"[[[0.0003632245061453432, -0.9938905835151672,...",[[0.7888966]]


In [ ]:
top_10_similar.reset_format()

In [ ]:
def generate_next_response(prompt):
  scores = top_10_similar.map(lambda row: score(row, prompt))
  result = scores.sort('score')

  return result[-1][prompt]

In [ ]:
past_sentences[1]="mama"

In [ ]:
generate_next_response("Would you like to attempt protocols for joy?")

  0%|          | 0/5 [00:00<?, ?ex/s]

[0.30211480362537757, 0.33232628398791536, 0.3655589123867069]
Я очень рад, что вы чувствуете себя прекрасно. В связи с этим я бы посоветовал вам изучить программу дистанционного обучения для того направления которое вам будет интересно. has a novelty score 1.0 and fluency score 0.013663751161975423 amd empathy_score1.0 
[0.30211480362537757, 0.33232628398791536, 0.3655589123867069]
Я очень рад слышать что у вас сегодня все отлично. У меня есть для вас интересное предложения, которое однозначно пойдет вам на пользу has a novelty score 1.0 and fluency score 0.030006694409541915 amd empathy_score1.0 
[0.30211480362537757, 0.33232628398791536, 0.3655589123867069]
Рада, что вы на такой позитивной ноте сегодня. Хотелось бы посоветовать вам одно упражнение. has a novelty score 1.0 and fluency score 0.03803148140705487 amd empathy_score1.0 
[0.30211480362537757, 0.33232628398791536, 0.3655589123867069]
Воспользуйтесь методикой из программы has a novelty score 1.0 and fluency score 0.000189525

'Я рад, что у вас все наладилось после той ситуации. Я могу вам посоветовать одну из дыхательных практик на случай тревоги.'

# Function to select best utterance

In [ ]:
from torch.utils.data.dataset import Dataset
def get_dataset(emotion):
  if emotion == "sad":
    dataset = sad_dialogue
  return dataset

In [ ]:
def get_response(emotion, situation):
  for 

In [ ]:
def score(row):
  score = row["ppl_how_recent_is_the_event?"]
  return {'score': score}

In [ ]:
top_10_similar.map(score)

  0%|          | 0/11 [00:00<?, ?ex/s]

Dataset({
    features: ['age', 'sad_utterance', 'how_recent_is_the_event?', 'specific_event_trigger', 'additional_questions_permission', 'feeling_envy_mistrust_trigger', 'method_6', 'self_blame_trigger', 'gender', 'embeddings', 'acd', 'ppl_how_recent_is_the_event?', 'ppl_specific_event_trigger', 'ppl_additional_questions_permission', 'ppl_feeling_envy_mistrust_trigger', 'ppl_method_6', 'similarity', 'score'],
    num_rows: 11
})

In [ ]:
# choosing a random sentence
top_10.sample(n=3)['specific_event_trigger']

44    Вижу, что вам не очень хорошо, спасибо вам за ...
48    Мне очень жаль, что вам так плохо. Ваша грусть...
68    Мне жаль, что у тебя плохое настроение. Думаю,...
Name: specific_event_trigger, dtype: object

## Fluency

In [ ]:
corpus_embeddings = epru['train']['embeddings'].to('cuda')
corpus_embeddings = util.normalize_embeddings(corpus_embeddings)

# query_embeddings = query_embeddings.to('cuda')
# query_embeddings = util.normalize_embeddings(query_embeddings)
# hits = util.semantic_search(query_embeddings, corpus_embeddings, score_function=util.dot_score)

AttributeError: ignored

In [ ]:
epru.set_format('torch', columns=['embeddings'])

In [ ]:
corpus_embeddings = epru['train']['embeddings'][0][0]
corpus_embeddings = util.normalize_embeddings(corpus_embeddings)


AttributeError: ignored

In [ ]:
query = "рудг"
query_embedding = pipe(query)
querry_embedding[0][0].shape

AttributeError: ignored

In [ ]:
util.cos_sim(query_embedding[0][0], epru['train']['embeddings'][100][0][0])

tensor([[0.3523]])

In [ ]:
epru['train']['embeddings'][100][0][0].shape

(768,)